In [1]:
import os
import pandas as pd
from Data_loader import Dataset_Custom
import argparse
import warnings
from tools import EarlyStopping
from torch.utils.data import Dataset, DataLoader
from utils import get_data
from Model import ANN
from torch import nn
import torch
import copy
from tqdm import tqdm
import random 
import numpy as np
warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(0)

In [3]:
from Server import  Server
from Clients import Client
from Train import Trainer

In [4]:
parser_train = argparse.ArgumentParser(description='FL')
parser_train.add_argument('--root_path', type=str, default='../Data/GFC12/')
parser_train.add_argument('--dataset_paths', type=list, default=["wf1", "wf2", "wf3", "wf4", "wf5", "wf6", "wf7"])
parser_train.add_argument('--number_clients', type=int, default=7)
parser_train.add_argument('--seq_len', type=int, default=24*4)
parser_train.add_argument('--pred_len', type=int, default=2)
parser_train.add_argument('--label_len', type=int, default=0)
parser_train.add_argument('--train_length', type=int, default=16800)
parser_train.add_argument('--target', type=str, default='target')
parser_train.add_argument('--scale', type=bool, default=True)
parser_train.add_argument('--inverse', type=bool, default=True)

parser_train.add_argument('--lr', type=float, default=1e-4)
parser_train.add_argument('--global_epochs', type=int, default=200)
parser_train.add_argument('--local_epochs', type=int, default=1)
parser_train.add_argument('--fine_tune_epochs', type=int, default=20)
parser_train.add_argument('--patience', type=int, default=3)
parser_train.add_argument('--fed_patience', type=int, default=3)
parser_train.add_argument('--hidden_layers', type=list, default=[64,64,64])
parser_train.add_argument('--input_size', type=int, default=293)
parser_train.add_argument('--output_size', type=int, default=9)

parser_train.add_argument('--fine_tune_lr', type=float, default=1e-5)
parser_train.add_argument('--ensemble_flag', type=bool, default=True)
parser_train.add_argument('--batch_size', type=int, default=64)
parser_train.add_argument('--device', type=str, default='cuda:2' if torch.cuda.is_available() else 'cpu')
parser_train.add_argument('--forecasting_mode', type=str, default='prob')
parser_train.add_argument('--model_type', type=str, default='NN')
parser_train.add_argument('--model_save_path', type=str, default='../Model12/', help='location of model checkpoints')
parser_train.add_argument('--quantiles', type=list, default=[0.1,0.2,0.3,0.4, 0.5,0.6,0.7,0.8, 0.9])
args_train = parser_train.parse_args(args=[])


In [5]:
seed_everything(0)

In [6]:
clients=[]
for path in tqdm(args_train.dataset_paths):
    args_temp=copy.deepcopy(args_train)
    args_temp.dataset_paths=path
    clients.append(Client(args_temp))

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:05<00:00,  1.31it/s]


In [7]:
server = Server(args_train,clients)

In [8]:
server.fed_train()

test performance: [0.1678826533870338, 0.1676450884097243, 0.19594906395530864, 0.18001982141030978, 0.1723320048863757, 0.18455327748742006, 0.18167505989948365]
Epoch: 0 | Loss: 0.0850
Epoch: 0 | Loss: 0.0630
Epoch: 0 | Loss: 0.1112
Epoch: 0 | Loss: 0.0855
Epoch: 0 | Loss: 0.0872
Epoch: 0 | Loss: 0.0703
Epoch: 0 | Loss: 0.0827
Federated training Epoch [1/200] Val Loss: 0.0716
test performance: [0.07541269663568229, 0.08067317516224025, 0.0936548399853788, 0.08484051999163954, 0.0873298574851392, 0.0839765701851208, 0.08878994756059287]
Epoch: 0 | Loss: 0.0500
Epoch: 0 | Loss: 0.0589
Epoch: 0 | Loss: 0.0773
Epoch: 0 | Loss: 0.0580
Epoch: 0 | Loss: 0.0551
Epoch: 0 | Loss: 0.0589
Epoch: 0 | Loss: 0.0712
Federated training Epoch [2/200] Val Loss: 0.0560
test performance: [0.05982974134640743, 0.06668905890865685, 0.07294007199369881, 0.06689255068410341, 0.06950944847082846, 0.06527284491960317, 0.07013873660248028]
Epoch: 0 | Loss: 0.0509
Epoch: 0 | Loss: 0.0518
Epoch: 0 | Loss: 0.0590


ANN(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=293, out_features=64, bias=True)
    (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
  )
  (relu): ReLU()
  (output): Linear(in_features=64, out_features=9, bias=True)
)

In [9]:
server.central_train()

Epoch [1/200] Train Loss: 0.0883 Val Loss: 0.0707
Epoch [2/200] Train Loss: 0.0658 Val Loss: 0.0569
Epoch [3/200] Train Loss: 0.0517 Val Loss: 0.0459
Epoch [4/200] Train Loss: 0.0447 Val Loss: 0.0411
Epoch [5/200] Train Loss: 0.0413 Val Loss: 0.0386
Epoch [6/200] Train Loss: 0.0392 Val Loss: 0.0372
Epoch [7/200] Train Loss: 0.0378 Val Loss: 0.0372
Epoch [8/200] Train Loss: 0.0369 Val Loss: 0.0353
Epoch [9/200] Train Loss: 0.0363 Val Loss: 0.0349
Epoch [10/200] Train Loss: 0.0355 Val Loss: 0.0338
Epoch [11/200] Train Loss: 0.0351 Val Loss: 0.0333
Epoch [12/200] Train Loss: 0.0349 Val Loss: 0.0332
Epoch [13/200] Train Loss: 0.0343 Val Loss: 0.0336
Epoch [14/200] Train Loss: 0.0342 Val Loss: 0.0324
Epoch [15/200] Train Loss: 0.0338 Val Loss: 0.0328
Epoch [16/200] Train Loss: 0.0336 Val Loss: 0.0339
Epoch [17/200] Train Loss: 0.0336 Val Loss: 0.0319
Epoch [18/200] Train Loss: 0.0334 Val Loss: 0.0319
Epoch [19/200] Train Loss: 0.0332 Val Loss: 0.0320
Epoch [20/200] Train Loss: 0.0329 Val Lo

ANN(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=293, out_features=64, bias=True)
    (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
  )
  (relu): ReLU()
  (output): Linear(in_features=64, out_features=9, bias=True)
)

In [10]:
server.local_train()

Launch Local Training!
Epoch [1/200] Train Loss: 0.0856 Val Loss: 0.0683
Epoch [2/200] Train Loss: 0.0610 Val Loss: 0.0506
Epoch [3/200] Train Loss: 0.0481 Val Loss: 0.0432
Epoch [4/200] Train Loss: 0.0423 Val Loss: 0.0419
Epoch [5/200] Train Loss: 0.0395 Val Loss: 0.0371
Epoch [6/200] Train Loss: 0.0379 Val Loss: 0.0357
Epoch [7/200] Train Loss: 0.0368 Val Loss: 0.0349
Epoch [8/200] Train Loss: 0.0359 Val Loss: 0.0341
Epoch [9/200] Train Loss: 0.0353 Val Loss: 0.0348
Epoch [10/200] Train Loss: 0.0351 Val Loss: 0.0339
Epoch [11/200] Train Loss: 0.0346 Val Loss: 0.0327
Epoch [12/200] Train Loss: 0.0341 Val Loss: 0.0326
Epoch [13/200] Train Loss: 0.0340 Val Loss: 0.0325
Epoch [14/200] Train Loss: 0.0340 Val Loss: 0.0326
Epoch [15/200] Train Loss: 0.0338 Val Loss: 0.0323
Epoch [16/200] Train Loss: 0.0333 Val Loss: 0.0323
Epoch [17/200] Train Loss: 0.0331 Val Loss: 0.0337
Epoch [18/200] Train Loss: 0.0331 Val Loss: 0.0319
Epoch [19/200] Train Loss: 0.0329 Val Loss: 0.0332
Epoch [20/200] Tr

In [11]:
local_fine_tune_losses=[]
local_fine_tune_preds=[]
local_fine_tune_models=[]
seed_everything(1)
for i in range(args_train.number_clients):
    local_fine_tune_pred,local_fine_tune_loss,local_fine_tune_model=clients[i].local_fine_tune(fine_tune_epochs=30)
    local_fine_tune_losses.append(local_fine_tune_loss)
    local_fine_tune_preds.append(local_fine_tune_pred)
    local_fine_tune_models.append(local_fine_tune_model)
print(local_fine_tune_losses)

1e-05 0
Epoch [1/30] Train Loss: 0.0318 Val Loss: 0.0306
Epoch [2/30] Train Loss: 0.0320 Val Loss: 0.0306
Epoch [3/30] Train Loss: 0.0319 Val Loss: 0.0306
Epoch [4/30] Train Loss: 0.0319 Val Loss: 0.0305
Epoch [5/30] Train Loss: 0.0318 Val Loss: 0.0304
Epoch [6/30] Train Loss: 0.0318 Val Loss: 0.0306
Epoch [7/30] Train Loss: 0.0318 Val Loss: 0.0304
Epoch [8/30] Train Loss: 0.0318 Val Loss: 0.0305
Epoch [9/30] Train Loss: 0.0318 Val Loss: 0.0304
Epoch [10/30] Train Loss: 0.0317 Val Loss: 0.0304
Epoch [11/30] Train Loss: 0.0317 Val Loss: 0.0304
Epoch [12/30] Train Loss: 0.0317 Val Loss: 0.0304
Epoch [13/30] Train Loss: 0.0317 Val Loss: 0.0304
Epoch [14/30] Train Loss: 0.0317 Val Loss: 0.0304
Epoch [15/30] Train Loss: 0.0317 Val Loss: 0.0305
Epoch [16/30] Train Loss: 0.0317 Val Loss: 0.0303
Epoch [17/30] Train Loss: 0.0317 Val Loss: 0.0304
Epoch [18/30] Train Loss: 0.0317 Val Loss: 0.0303
Epoch [19/30] Train Loss: 0.0317 Val Loss: 0.0303
Epoch [20/30] Train Loss: 0.0316 Val Loss: 0.0304
E

In [25]:
'''import utils import plot_prob_result
args_temp=copy.deepcopy(args_train)
args_temp.dataset_paths='wf7'
test_data, test_loader = get_data(args_train,flag='test')
actual_y=[]
for idx, (seq_x, seq_x_concat, seq_y) in enumerate(test_loader):
    actual_y.append(seq_y)
actual_y = torch.cat([torch.flatten(t) for t in actual_y])'''

In [12]:
fed_local_losses=[]
fed_local_preds=[]
fed_local_models=[]
for i in range(args_train.number_clients):
    fed_local_pred,fed_local_loss,fed_local_model=clients[i].fed_local_evaluation()
    fed_local_losses.append(fed_local_loss)
    fed_local_preds.append(fed_local_pred)
    fed_local_models.append(fed_local_model)
print(fed_local_losses)
print(np.mean(fed_local_losses))

[0.03369925597886721, 0.03519536803591333, 0.040536052285859436, 0.03649028574996819, 0.03685282969732501, 0.035751149217815026, 0.037361839212068954]
0.03655525431111674


In [13]:
local_losses=[]
local_preds=[]
local_models=[]
for i in range(args_train.number_clients):
    local_pred,local_loss,local_model=clients[i].local_evaluation()
    local_losses.append(local_loss)
    local_preds.append(local_pred)
    local_models.append(local_model)
print(local_losses)

[0.03371577699706979, 0.03496455265949034, 0.04149482667139948, 0.03731695264705444, 0.03818863684834581, 0.03602887266506887, 0.03983221346933446]


In [14]:
central_losses=[]
central_preds=[]
central_models=[]
for i in range(args_train.number_clients):
    central_pred,central_loss,central_model=server.central_evaluation(dataset=i)
    central_losses.append(central_loss)
    central_preds.append(central_pred)
    central_models.append(central_model)
print(central_losses)

[0.03443343698187438, 0.03807613101775107, 0.043018698928341884, 0.03778318985531183, 0.040288623243774455, 0.03716161004459, 0.04036670377197331]


In [15]:
print(local_losses)
print(central_losses)
print(fed_local_losses)
print(local_fine_tune_losses)

[0.03371577699706979, 0.03496455265949034, 0.04149482667139948, 0.03731695264705444, 0.03818863684834581, 0.03602887266506887, 0.03983221346933446]
[0.03443343698187438, 0.03807613101775107, 0.043018698928341884, 0.03778318985531183, 0.040288623243774455, 0.03716161004459, 0.04036670377197331]
[0.03369925597886721, 0.03519536803591333, 0.040536052285859436, 0.03649028574996819, 0.03685282969732501, 0.035751149217815026, 0.037361839212068954]
[0.033156270237818156, 0.034703732302335845, 0.040098275313128345, 0.03563182553187115, 0.0366086190126275, 0.0351469776567633, 0.03705518918226741]


In [16]:
df = pd.DataFrame({
    'local_losses': local_losses,
    'central_losses': central_losses,
    'fed_local_losses': fed_local_losses,
    'local_fine_tune_losses': local_fine_tune_losses
})
df.T.to_csv('losses.csv', index=False)

In [17]:
actual_y_lst=[]

for i in range(args_train.number_clients):
    args_temp=copy.deepcopy(args_train)
    args_temp.dataset_paths='wf'+str(i+1)
    test_data, test_loader = get_data(args_temp,flag='test')
    actual_y=[]
    for idx, (seq_x, seq_x_concat, seq_y) in enumerate(test_loader):
        actual_y.append(seq_y)
    actual_y = torch.cat([torch.flatten(t) for t in actual_y])
    actual_y_lst.append(actual_y)


In [18]:
import pickle

# Save the server object
with open('../result/2/server_benchmark.pkl', 'wb') as f:
    pickle.dump(server, f)

# Save the clients object
with open('../result/2/clients_benchmark.pkl', 'wb') as f:
    pickle.dump(clients, f)

In [334]:
import pickle

# Save server and clients
with open('server.pkl', 'wb') as f:
    pickle.dump(server, f)

with open('clients.pkl', 'wb') as f:
    pickle.dump(clients, f)

# Load server and clients
with open('server.pkl', 'rb') as f:
    server = pickle.load(f)

with open('clients.pkl', 'rb') as f:
    clients = pickle.load(f)